# Wine Space - Google Colab 运行脚本

这个notebook用于在Google Colab上运行wine-spave项目。

**GitHub仓库**: https://github.com/YifeiCAO/wine-spave

## 使用说明

1. 在Google Colab中打开这个notebook
2. 运行所有cell（按顺序）
3. 项目会自动克隆、安装依赖并运行训练

**注意**: 确保在Colab中启用GPU（运行时 -> 更改运行时类型 -> GPU）


## 1. 检查GPU和安装基础依赖


In [ ]:
# 检查GPU
import torch
print("PyTorch版本:", torch.__version__)
print("CUDA可用:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU设备:", torch.cuda.get_device_name(0))
    print("CUDA版本:", torch.version.cuda)
else:
    print("警告: 未检测到GPU，建议在Colab中启用GPU加速")

# 安装必要的依赖（Colab可能没有的）
%pip install -q statsmodels seaborn==0.11.1


## 2. 克隆GitHub仓库


In [ ]:
# 克隆仓库
import os
import shutil

repo_url = "https://github.com/YifeiCAO/wine-spave.git"
repo_name = "wine-spave"

# 如果已经存在，先删除
if os.path.exists(repo_name):
    shutil.rmtree(repo_name)

# 克隆仓库
os.system(f"git clone {repo_url}")

# 进入项目目录
os.chdir(repo_name)
print(f"当前工作目录: {os.getcwd()}")

# 列出文件
print("\n项目文件:")
os.system("ls -la")


## 3. 安装项目依赖


In [ ]:
# 安装requirements.txt中的依赖
%pip install -q -r requirements.txt

# 验证安装
import sys
sys.path.insert(0, os.getcwd())

print("依赖安装完成！")


## 4. 导入项目模块


In [ ]:
# 导入项目模块
import torch
import torch.nn as nn
import numpy as np
import random
from torch.utils.data import DataLoader

# 导入项目模块
from data import get_loaders, GridDataGenerator, GridDataset, grid_collate
from models import get_model

print("模块导入完成！")


## 5. 设置参数和配置


In [ ]:
# 创建参数类
class Args:
    def __init__(self):
        # 设备设置 - Colab使用GPU
        self.use_cuda = True
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.seed = 0
        
        # 数据集设置
        self.use_images = False  # 使用索引而不是图像（更快）
        self.image_dir = 'images/faces16'
        self.training_regime = 'ungrouped'  # 使用ungrouped模式
        self.grid_size = 4
        self.ctx_order = 'first'
        self.inner_4x4 = False
        
        # 训练设置
        self.bs = 32  # Batch size
        self.lr = 0.001
        self.n_steps = 500  # 训练步数（可以根据需要调整）
        self.print_every = 50
        self.test_every = 100
        self.analyze_every = 200
        
        # 模型设置
        self.model_name = 'rnn'
        self.ctx_scale = 1.0
        self.measure_grad_norm = False
        
        # 分析设置
        self.dim_red_method = 'pca'
        
        # 运行设置
        self.n_runs = 1  # Colab上先运行1次测试

# 创建参数对象
args = Args()

# 设置随机种子
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)

print("参数设置完成！")
print(f"设备: {args.device}")
print(f"模型: {args.model_name}")
print(f"训练步数: {args.n_steps}")
print(f"Batch size: {args.bs}")
print(f"学习率: {args.lr}")


## 6. 测试数据生成


In [ ]:
# 测试数据生成
print("生成数据...")
grid = GridDataGenerator(
    training_regime=args.training_regime,
    size=args.grid_size,
    use_images=args.use_images,
    image_dir=args.image_dir,
    inner_4x4=args.inner_4x4
)

print(f"✓ 训练样本数: {len(grid.train)}")
print(f"✓ 测试样本数: {len(grid.test)}")
print(f"✓ 分析样本数: {len(grid.analyze)}")

# 查看一个样本
sample = grid.train[0]
ctx, loc1, loc2, y, info = sample
print(f"\n示例样本:")
print(f"  上下文: {ctx}, 位置1: {loc1}, 位置2: {loc2}, 标签: {y}")


## 7. 创建数据加载器


In [ ]:
# 使用get_loaders获取数据加载器
data = get_loaders(args)
train_loader, test_loader, analyze_loader = data

print("✓ 数据加载器创建完成")
print(f"  训练batches: {len(train_loader)}")
print(f"  测试batches: {len(test_loader)}")

# 测试一个batch
batch = next(iter(train_loader))
ctx, f1, f2, y, info = batch
print(f"\nBatch形状:")
print(f"  上下文: {ctx.shape}, 葡萄酒1: {f1.shape}, 葡萄酒2: {f2.shape}, 标签: {y.shape}")


## 8. 创建模型


In [ ]:
# 创建模型
print("创建模型...")
model = get_model(args)
model.to(args.device)

# 打印模型信息
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✓ 模型创建完成: {args.model_name}")
print(f"  总参数数: {total_params:,}")
print(f"  可训练参数: {trainable_params:,}")
print(f"  设备: {next(model.parameters()).device}")

# 测试前向传播
model.eval()
with torch.no_grad():
    test_ctx = ctx[:4].to(args.device)
    test_f1 = f1[:4].to(args.device)
    test_f2 = f2[:4].to(args.device)
    
    y_hat, reps = model(test_ctx, test_f1, test_f2)
    print(f"\n✓ 前向传播测试成功")
    print(f"  输出形状: {y_hat.shape}")


## 9. 训练模型


In [ ]:
# 设置训练组件
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
loss_fn = nn.CrossEntropyLoss()

print("开始训练...")
print(f"训练步数: {args.n_steps}")
print(f"每 {args.print_every} 步打印一次损失\n")

model.train()
losses = []
accuracies = []

for step, batch in enumerate(train_loader):
    if step >= args.n_steps:
        break
    
    optimizer.zero_grad()
    
    ctx, f1, f2, y, info = batch
    ctx = ctx.to(args.device)
    f1 = f1.to(args.device)
    f2 = f2.to(args.device)
    y = y.to(args.device)
    
    # 前向传播
    y_hat, _ = model(ctx, f1, f2)
    loss = loss_fn(y_hat, y)
    
    # 反向传播
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    # 计算准确率
    preds = torch.argmax(y_hat, dim=1)
    acc = (preds == y).float().mean().item()
    accuracies.append(acc)
    
    # 打印进度
    if (step + 1) % args.print_every == 0:
        avg_loss = np.mean(losses[-args.print_every:])
        avg_acc = np.mean(accuracies[-args.print_every:])
        print(f"Step {step+1}/{args.n_steps} | Loss: {avg_loss:.4f} | Acc: {avg_acc:.4f}")

print(f"\n✓ 训练完成！")
print(f"  最终损失: {losses[-1]:.4f}")
print(f"  最终准确率: {accuracies[-1]:.4f}")


## 10. 测试模型


In [ ]:
# 导入test函数
from test import test

print("测试模型...")
test_results = test(model, test_loader, args)

print(f"\n✓ 测试完成！")
print(f"  总体准确率: {test_results['acc']:.4f}")
print(f"  一致准确率: {test_results['cong_acc']:.4f}")
print(f"  不一致准确率: {test_results['incong_acc']:.4f}")
print(f"\n上下文0各等级准确率:")
for i, acc in enumerate(test_results['loc1_ctx0_acc']):
    print(f"    等级 {i}: {acc:.4f}")
print(f"\n上下文1各等级准确率:")
for i, acc in enumerate(test_results['loc1_ctx1_acc']):
    print(f"    等级 {i}: {acc:.4f}")


## 11. 可视化训练过程


In [ ]:
# 可视化训练过程
import matplotlib.pyplot as plt

# 绘制损失曲线
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# 损失曲线
ax1.plot(losses, alpha=0.6, label='Raw Loss')
if len(losses) >= 10:
    window = 10
    moving_avg = np.convolve(losses, np.ones(window)/window, mode='valid')
    ax1.plot(range(window-1, len(losses)), moving_avg, label=f'Moving Avg (window={window})', linewidth=2)
ax1.set_xlabel('Step')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 准确率曲线
ax2.plot(accuracies, alpha=0.6, label='Raw Accuracy')
if len(accuracies) >= 10:
    window = 10
    moving_avg = np.convolve(accuracies, np.ones(window)/window, mode='valid')
    ax2.plot(range(window-1, len(accuracies)), moving_avg, label=f'Moving Avg (window={window})', linewidth=2)
ax2.set_xlabel('Step')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ 可视化完成")


## 12. 使用完整训练函数（可选）

如果你想使用完整的训练函数（包含定期测试和分析），可以运行下面的cell。


In [ ]:
# 使用完整的train函数（包含测试和分析）
from train import train

print("使用完整训练函数...")
print("注意: 这会运行完整的训练循环，包括定期测试和分析\n")

# 重新创建模型和数据
model_full = get_model(args)
model_full.to(args.device)
data_full = get_loaders(args)

# 运行完整训练
results, analyses = train(0, model_full, data_full, args)

print(f"\n✓ 完整训练完成！")
print(f"  训练损失记录数: {len(results['train_losses'])}")
print(f"  训练准确率记录数: {len(results['train_accs'])}")
print(f"  测试准确率记录数: {len(results['test_accs'])}")

if results['train_accs']:
    print(f"\n最终准确率:")
    print(f"  训练: {results['train_accs'][-1]['acc']:.4f}")
    print(f"  测试: {results['test_accs'][-1]['acc']:.4f}")


## 总结

✓ 项目已成功在Colab上运行！

**完成的任务:**
1. ✅ 克隆GitHub仓库
2. ✅ 安装依赖
3. ✅ 数据生成和加载
4. ✅ 模型创建
5. ✅ 模型训练
6. ✅ 模型测试
7. ✅ 结果可视化

**提示:**
- 可以在第5步调整训练参数（n_steps, lr, bs等）
- 可以修改模型类型（model_name）
- 训练结果会自动保存在内存中，可以进一步分析

**保存结果:**
如果需要保存结果，可以运行：
```python
import pickle
with open('results.pkl', 'wb') as f:
    pickle.dump({'losses': losses, 'accuracies': accuracies, 'test_results': test_results}, f)
```
